Obtener la ruta de los datasets

In [1]:
# Add your PRIVATE credentials
# Do not use "!export KAGGLE_USERNAME= ..." OR "" around your credential
%env KAGGLE_USERNAME=''
%env KAGGLE_KEY=''

# Verify
!export -p | grep KAGGLE_USERNAME
!export -p | grep KAGGLE_KEY

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
## Cargar archivos de la competencia

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/tp-n2-aprendizaje-profundo-2021-by-datitos-v2/fifa2021_training.csv')
df_infer = pd.read_csv('/kaggle/input/tp-n2-aprendizaje-profundo-2021-by-datitos-v2/fifa2021_test.csv')

Análisis exploratorio del dataset

In [1]:
df.iloc[0]

Particionar datasets

In [1]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, stratify=df.Position, train_size=0.9, random_state=42)

Definir transformaciones y columnas a utilizar

In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

variables_descartar = [
   'Position', # variable objetivo
    'ID',
    'Name',
    'Natinality',
    'Potential',
    'BirthDate',
    'Value',
    'Wage',
    'Club',
    'Club_KitNumber',
    'Club_JoinedClub',
    'Club_ContractLength',
]

variables_categóricas = df.drop(columns=variables_descartar).select_dtypes(include=np.object).columns
variables_numéricas   = df.drop(columns=variables_descartar).select_dtypes(include=np.number).columns

transformador = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),  variables_categóricas), # PreferredFoot, PlayerWorkRate, Sex resolver nacionalidad con handle_unknown='ignore' Eso trae otro probl despues, asi q por ahora vamos con eliminar los dos paises q me joden
    (StandardScaler(), variables_numéricas),   # Overal, Potential, Height, etc.
    remainder='drop' # descarta las columnas no mencionadas en las transformaciones
)


Entrenar transformador

Esencialmente, calculamos los promedios y las desviaciones estándares que usaremos para la estandarización.

 SOLO PARA EL DATASET DEtransformador.fit(df_train) ENTRENAMIENTO.

In [1]:
transformador.fit(df_train)

Transformar datasets

In [1]:
# Se agrega la columna a predecir en el dataset para que tenga las mismas columnas que df_train :/  
df_infer['Position'] = None


X_train = transformador.transform(df_train)
X_valid = transformador.transform(df_valid)
X_infer = transformador.transform(df_infer)

Transformar variable objetivo

Como la variable objetivo es del tipo string, hay que llevarla a un tipo numérico para que PyTorch pueda procesarla.

Este transformador mapea posiciones DEF, FWD, GK, MID a enteros y viceversa — la transformación inversa será útil para convertir las predicciones (enteros) en posiciones otra vez.

In [1]:
from sklearn.preprocessing import LabelEncoder

transformador_etiquetas = LabelEncoder()

transformador_etiquetas.fit(df_train.Position)

y_train = transformador_etiquetas.transform(df_train.Position)
y_valid = transformador_etiquetas.transform(df_valid.Position)

Instanciar Datasets de PyTorch

El aprendizaje profundo es especialmente efectivo para imágenes y texto; para datos tabulares (como un DataFrame) el aprendizaje de máquinas clásico suele funcionar bastante bien, de ahí que PyTorch no cuente con facilidades para tratar este tipo de problemas.

In [1]:
from torch.utils.data import Dataset

class Tabular(Dataset):
    def __init__(self, X, y=None):
        self.X = X.astype(np.float32) # soluciona "Expected object of scalar type Float but got scalar type Double"
        self.y = y 

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, item):
        if self.y is None:
            return self.X[item]
        else:
            return self.X[item], self.y[item]

        
ds_train = Tabular(X_train, y_train)

In [1]:
ds_train[10]

Instanciar DataLoaders de PyTorch

In [1]:
from torch.utils.data import DataLoader

dl_train = DataLoader(ds_train, batch_size=32, shuffle=False)

In [1]:
import torch
X_valid = torch.tensor(X_valid).float()
X_infer = torch.tensor(X_infer).float()
y_valid = torch.tensor(y_valid)

Instanciar modelo

In [1]:
import torch
import torch.nn as nn

In [1]:
IN  = X_train.shape[1]
OUT = len(transformador_etiquetas.classes_)

modelo = nn.Sequential(
    nn.Linear(IN,  8),
    nn.Linear( 8, 64), nn.ReLU(),
    nn.Linear(64, 32), nn.ReLU(), 
    nn.Linear(32, OUT)
)

In [1]:
weights = torch.tensor([0.2,0.2,0.2,0.2], dtype=torch.float)

criterio = nn.CrossEntropyLoss(weight=weights) #weight=weights 
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.01)

### Entrenar Modelo

In [1]:
from sklearn.metrics import balanced_accuracy_score

ÉPOCAS = 10

for época in range(ÉPOCAS):
    # activa capas Dropout, BatchNorm si las hubiese
    modelo.train()

    pérdidas_train = []
    
    for X_lote, y_lote in dl_train:
        optimizador.zero_grad()

        predicciones = modelo(X_lote)
        pérdida = criterio(predicciones, y_lote)

        pérdida.backward()
        optimizador.step()
        
        pérdidas_train.append(pérdida.item())
    
    # desactiva capas Dropout, BatchNorm si las hubiese
    modelo.eval()
    
    with torch.no_grad():
        predicciones = modelo(X_valid)
        pérdida = criterio(predicciones, y_valid)
        
        y_pred = predicciones.argmax(dim=1) # selecciona la clase con mayor probabilidad
        
        efectividad = balanced_accuracy_score(y_valid, y_pred)
    
    
    print(f'{época:3d}  |  Train loss: {np.mean(pérdidas_train):.3f}    Valid loss: {pérdida:.3f}    Valid accuracy: {efectividad:.2f}')
criterio = nn.CrossEntropyLoss(weight=weights) 
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.0001)

### Inferir datos de prueba

In [1]:
with torch.no_grad():
    y_infer = modelo(X_infer).argmax(dim=1)

df_infer['Position'] = transformador_etiquetas.inverse_transform(y_infer)

(
    df_infer[['ID', 'Position']]
    .rename(columns={'ID':'Id', 'Position':'Category'})
    .to_csv('submit.csv', index=False)
)

In [1]:
#!pip install kaggle

In [1]:
#!kaggle competitions submit -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2 -f submit.csv -m "Ultimo submit!"